<a href="https://colab.research.google.com/github/amalsalilan/B3-Developing-Named-Entity-Recognition-NER-Models-for-Financial-Data-Extraction-/blob/Isha/financial_ner_training_v3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [27]:
! pip install -U spacy -q

In [28]:

!python -m spacy info


============================== Info about spaCy ==============================

spaCy version    3.8.7                         
Location         /usr/local/lib/python3.12/dist-packages/spacy
Platform         Linux-6.6.105+-x86_64-with-glibc2.35
Python version   3.12.12                       
Pipelines        en_core_web_sm (3.8.0)        



In [29]:
import spacy
from spacy.tokens import DocBin
from tqdm import tqdm

nlp = spacy.blank("en") # load a new spacy model
db = DocBin() # create a DocBin object

In [30]:
import json
f = open('/content/annotations_i.json')
TRAIN_DATA = json.load(f)

In [31]:
TRAIN_DATA

{'classes': ['DATE', 'ORG', 'PERSON', 'MONEY', 'PLACE', 'PERCENTAGE'],
 'annotations': [['On March 12, 2025, BlueRock Capital Pvt. Ltd., headquartered in Mumbai, announced a ₹48.7 crore investment in Aurora FinTech Solutions, a Bangalore-based digital lending startup founded by Raghav Menon. The investment represents a 12.5% equity stake, making BlueRock one of Aurora’s major institutional backers. The company stated that the funding round was oversubscribed by nearly 40%, signaling strong investor confidence in Aurora’s AI-driven credit assessment platform.\r',
   {'entities': [[3, 17, 'DATE'],
     [19, 45, 'ORG'],
     [64, 70, 'PLACE'],
     [84, 95, 'MONEY'],
     [110, 134, 'ORG'],
     [138, 153, 'PLACE'],
     [189, 202, 'PERSON'],
     [231, 236, 'PERCENTAGE'],
     [258, 266, 'ORG'],
     [274, 282, 'ORG'],
     [383, 386, 'PERCENTAGE'],
     [428, 436, 'ORG']]}],
  None,
  None,
  ['According to Financial Insight Weekly, Aurora’s annual revenue grew from ₹26 crore in FY 2023

In [34]:
for item in tqdm(TRAIN_DATA['annotations']):
    if item is None:
        print("Skipping None item")
        continue
    text, annot = item
    doc = nlp.make_doc(text)
    ents = []
    for start, end, label in annot["entities"]:
        span = doc.char_span(start, end, label=label, alignment_mode="contract")
        if span is None:
            print(f"Skipping entity for: {text[start:end]}")
        else:
            ents.append(span)
    doc.ents = ents
    db.add(doc)
db.to_disk("./training_data.spacy")

100%|██████████| 13/13 [00:00<00:00, 2823.28it/s]

Skipping None item
Skipping None item
Skipping None item
Skipping None item
Skipping None item
Skipping None item
Skipping None item
Skipping None item


In [35]:
! python -m spacy init config config.cfg --lang en --pipeline ner --optimize efficiency

ℹ Generated config template specific for your use case
- Language: en
- Pipeline: ner
- Optimize for: efficiency
- Hardware: CPU
- Transformer: None
✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [36]:
! python -m spacy train config.cfg --output ./ --paths.train ./training_data.spacy --paths.dev ./training_data.spacy

ℹ Saving to output directory: .
ℹ Using CPU

=========================== Initializing pipeline ===========================
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'ner']
ℹ Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00     41.50    0.00    0.00    0.00    0.00
 18     200        359.24   2190.62  100.00  100.00  100.00    1.00
 36     400          0.12      0.11  100.00  100.00  100.00    1.00
 62     600          0.00      0.00  100.00  100.00  100.00    1.00
 97     800          0.00      0.00  100.00  100.00  100.00    1.00
140    1000          0.00      0.00  100.00  100.00  100.00    1.00
191    1200          0.00      0.00  100.00  100.00  100.00    1.00
256    1400          0.00      0.00  100.00  100.00  100.00    1.00
346    1600          0.00      0.00 

In [37]:
nlp_ner = spacy.load("/content/model-best")

In [38]:
doc = nlp_ner("Market analysts at MorganEast Research, based in Singapore, forecast that the Indian digital lending sector will surpass ₹1.8 lakh crore in transaction volume by 2027, with Aurora capturing approximately 6.4% market share.")

In [39]:
spacy.displacy.render(doc, style="ent", jupyter=True) # display in Jupyter